Feedforward Neural Network with Regularization
=============

In [1]:
from __future__ import print_function
import numpy as np
import random
import scipy.io as sio
import tensorflow as tf
from six.moves import cPickle as pickle

First load the data dumped by MATLAB (*.mat file):

In [2]:
# X_init_offset_cancelled = sio.loadmat('scraping/X_init_offset_cancelled_scraping.mat', struct_as_record=True)['X_init_offset_cancelled']
# X_init_offset_cancelled = sio.loadmat('scraping/Xioc_phasePSI_scraping.mat', struct_as_record=True)['Xioc_phasePSI']
#X_234_all
X_init_offset_cancelled_all= sio.loadmat('scraping/X_gauss_basis_func_scraping.mat', struct_as_record=True)['X_gauss_basis_func'].astype(np.float32)
# X_init_offset_cancelled = sio.loadmat('scraping/Xioc_PD_ratio_mean_3std_scraping.mat', struct_as_record=True)['Xioc_PD_ratio_mean_3std']
# Ct_target = sio.loadmat('scraping/Ct_target_scraping.mat', struct_as_record=True)['Ct_target']

#X_234
X_init_offset_cancelled = sio.loadmat('scraping/X_gauss_basis_func_scraping_elim_3_train.mat', struct_as_record=True)['X_gauss_basis_func_train'].astype(np.float32)
#Ct_target_234
Ct_target = sio.loadmat('scraping/Ct_target_filt_scraping_elim_3_train.mat', struct_as_record=True)['Ct_target_filt_train'].astype(np.float32)

# Dataset for Extrapolation Test
#X_5toend
X_extrapolate_test = sio.loadmat('scraping/X_gauss_basis_func_scraping_elim_3_test.mat', struct_as_record=True)['X_gauss_basis_func_test'].astype(np.float32)
#Ct_5toend
Ctt_extrapolate_test = sio.loadmat('scraping/Ct_target_filt_scraping_elim_3_test.mat', struct_as_record=True)['Ct_target_filt_test'].astype(np.float32)

# Dummy Data for learning simulation/verification:
# X_init_offset_cancelled = sio.loadmat('scraping/dummy_X.mat', struct_as_record=True)['X']
# Ct_target = sio.loadmat('scraping/dummy_Ct.mat', struct_as_record=True)['Ct']

Verify the dimensions are correct and shuffle the data (for Stochastic Gradient Descent (SGD)):

In [3]:
N_data_extrapolate_test = Ctt_extrapolate_test.shape[0]
permutation_extrapolate_test = np.random.permutation(N_data_extrapolate_test)
permutation_extrapolate_test_select = permutation_extrapolate_test[1:1000]
X_extrapt = X_extrapolate_test[permutation_extrapolate_test_select, :]
Ctt_extrapt = Ctt_extrapolate_test[permutation_extrapolate_test_select, :]

print('X_init_offset_cancelled.shape =', X_init_offset_cancelled.shape)
print('Ct_target.shape =', Ct_target.shape)

N_data = Ct_target.shape[0]
D_input = X_init_offset_cancelled.shape[1]
D_output = Ct_target.shape[1]
print('N_data   =', N_data)
print('D_input  =', D_input)
print('D_output =', D_output)

print('X_extrapolate_test.shape =', X_extrapolate_test.shape)
print('Ctt_extrapolate_test.shape =', Ctt_extrapolate_test.shape)

random.seed(38)
np.random.seed(38)

X_init_offset_cancelled = X_init_offset_cancelled
X_init_offset_cancelled_all = X_init_offset_cancelled_all

permutation = np.random.permutation(N_data)
X_shuffled = X_init_offset_cancelled[permutation,:]
Ct_target_shuffled = Ct_target[permutation,:]

fraction_train_dataset = 0.85
fraction_test_dataset  = 0.075

N_train_dataset = np.round(fraction_train_dataset * N_data).astype(int)
N_test_dataset = np.round(fraction_test_dataset * N_data).astype(int)
N_valid_dataset = N_data - N_train_dataset - N_test_dataset
print('N_train_dataset =', N_train_dataset)
print('N_valid_dataset =', N_valid_dataset)
print('N_test_dataset  =', N_test_dataset)

X_train_dataset = X_shuffled[0:N_train_dataset,:]
Ct_train = Ct_target_shuffled[0:N_train_dataset,:]
X_valid_dataset = X_shuffled[N_train_dataset:(N_train_dataset+N_valid_dataset),:]
Ct_valid = Ct_target_shuffled[N_train_dataset:(N_train_dataset+N_valid_dataset),:]
X_test_dataset = X_shuffled[(N_train_dataset+N_valid_dataset):N_data,:]
Ct_test = Ct_target_shuffled[(N_train_dataset+N_valid_dataset):N_data,:]

X_init_offset_cancelled.shape = (198563, 250)
Ct_target.shape = (198563, 6)
N_data   = 198563
D_input  = 250
D_output = 6
X_extrapolate_test.shape = (128105, 250)
Ctt_extrapolate_test.shape = (128105, 6)
N_train_dataset = 168779
N_valid_dataset = 14892
N_test_dataset  = 14892


In [4]:
def computeNMSE(predictions, labels):
    mse = np.mean(np.square(predictions-labels), axis=0);
    var_labels = np.var(labels, axis=0)
    nmse = np.divide(mse, var_labels)
    return (nmse)

---
Feed-Forward Neural Network Model
---------

Here it goes:
 
 ---


In [5]:
import os

batch_size = 64
num_steps  = 700001

# Number of units in hidden layer
N_HIDDEN1_UNITS = 125
N_HIDDEN2_UNITS = 64
N_HIDDEN3_UNITS = 32

# L2 Regularizer constant
beta1 = 0.0000000001

logs_path = "/tmp/ffnn/"

def defineFeedForwardNeuralNetworkModel(input_size, num_hidden1_units, num_hidden2_units, num_hidden3_units, output_size):
    # Hidden 1 Layer
    with tf.variable_scope('hidden1', reuse=False):
        weights = tf.get_variable('weights', [input_size, num_hidden1_units], initializer=tf.random_normal_initializer(0.0, 1e-7))
        biases = tf.get_variable('biases', [num_hidden1_units], initializer=tf.constant_initializer(0))
    # Hidden 2 Layer
    with tf.variable_scope('hidden2', reuse=False):
        weights = tf.get_variable('weights', [num_hidden1_units, num_hidden2_units], initializer=tf.random_normal_initializer(0.0, 1e-7))
        biases = tf.get_variable('biases', [num_hidden2_units], initializer=tf.constant_initializer(0))
    # Hidden 3 Layer
    with tf.variable_scope('hidden3', reuse=False):
        weights = tf.get_variable('weights', [num_hidden2_units, num_hidden3_units], initializer=tf.random_normal_initializer(0.0, 1e-7))
        biases = tf.get_variable('biases', [num_hidden3_units], initializer=tf.constant_initializer(0))
    # Linear (Output) Layer
    with tf.variable_scope('linear', reuse=False):
        weights = tf.get_variable('weights', [num_hidden3_units, output_size], initializer=tf.random_normal_initializer(0.0, 1e-7))
        biases = tf.get_variable('biases', [output_size], initializer=tf.constant_initializer(0))
    
    return None

# Build prediction graph.
def performFeedForwardNeuralNetworkPrediction(train_dataset, input_size, num_hidden1_units, num_hidden2_units, num_hidden3_units, output_size, dropout_keep_prob):
    """Build the Feed-Forward Neural Network model for prediction.
    Args:
        train_dataset: training dataset's placeholder.
        num_hidden1_units: Size of the 1st hidden layer.
    Returns:
        outputs: Output tensor with the computed logits.
    """
    # Hidden 1
    with tf.variable_scope('hidden1', reuse=True):
        weights = tf.get_variable('weights', [input_size, num_hidden1_units])
        biases = tf.get_variable('biases', [num_hidden1_units])
        hidden1 = tf.nn.relu(tf.matmul(train_dataset, weights) + biases)
#         hidden1 = tf.matmul(train_dataset, weights) + biases
        hidden1_drop = tf.nn.dropout(hidden1, dropout_keep_prob)
    # Hidden 2
    with tf.variable_scope('hidden2', reuse=True):
        weights = tf.get_variable('weights', [num_hidden1_units, num_hidden2_units])
        biases = tf.get_variable('biases', [num_hidden2_units])
        hidden2 = tf.nn.relu(tf.matmul(hidden1_drop, weights) + biases)
        hidden2_drop = tf.nn.dropout(hidden2, dropout_keep_prob)
    # Hidden 3
    with tf.variable_scope('hidden3', reuse=True):
        weights = tf.get_variable('weights', [num_hidden2_units, num_hidden3_units])
        biases = tf.get_variable('biases', [num_hidden3_units])
        hidden3 = tf.nn.relu(tf.matmul(hidden2_drop, weights) + biases)
        hidden3_drop = tf.nn.dropout(hidden3, dropout_keep_prob)
    # Linear (Output)
    with tf.variable_scope('linear', reuse=True):
        weights = tf.get_variable('weights', [num_hidden3_units, output_size])
        biases = tf.get_variable('biases', [output_size])
        outputs = tf.matmul(hidden3_drop, weights) + biases
    
    return outputs

# Build training graph.
def performFeedForwardNeuralNetworkTraining(outputs, labels, initial_learning_rate, input_size, num_hidden1_units, num_hidden2_units, num_hidden3_units, output_size):
    """Build the training graph.
    
    Args:
        outputs: Output tensor, float - [BATCH_SIZE, output_size].
        labels : Labels tensor, float - [BATCH_SIZE, output_size].
        initial_learning_rate: The initial learning rate to use for gradient descent.
    Returns:
        train_op: The Op for training.
        loss: The Op for calculating loss.
    """
    # Create an operation that calculates L2 prediction loss.
    pred_l2_loss = tf.nn.l2_loss(outputs - labels, name='my_pred_l2_loss')
    
    # Create an operation that calculates L2 loss.
    # Hidden 1
    with tf.variable_scope('hidden1', reuse=True):
        weights = tf.get_variable('weights', [input_size, num_hidden1_units])
        biases = tf.get_variable('biases', [num_hidden1_units])
        hidden1_layer_l2_loss = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    # Hidden 2
    with tf.variable_scope('hidden2', reuse=True):
        weights = tf.get_variable('weights', [num_hidden1_units, num_hidden2_units])
        biases = tf.get_variable('biases', [num_hidden2_units])
        hidden2_layer_l2_loss = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    # Hidden 3
    with tf.variable_scope('hidden3', reuse=True):
        weights = tf.get_variable('weights', [num_hidden2_units, num_hidden3_units])
        biases = tf.get_variable('biases', [num_hidden3_units])
        hidden3_layer_l2_loss = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    # Linear (Output)
    with tf.variable_scope('linear', reuse=True):
        weights = tf.get_variable('weights', [num_hidden3_units, output_size])
        biases = tf.get_variable('biases', [output_size])
        output_layer_l2_loss = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    
    loss = tf.reduce_mean(pred_l2_loss, name='my_pred_l2_loss_mean') + (beta1 * (hidden1_layer_l2_loss + hidden2_layer_l2_loss + hidden3_layer_l2_loss + output_layer_l2_loss))
    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)
    # Exponentially-decaying learning rate:
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, num_steps, 0.1)
    # Create the gradient descent optimizer with the given learning rate.
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
#     train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
#     train_op = tf.train.MomentumOptimizer(learning_rate, momentum=learning_rate/4.0, use_nesterov=True).minimize(loss, global_step=global_step)
    train_op = tf.train.AdagradOptimizer(initial_learning_rate).minimize(loss, global_step=global_step)
    
    return train_op, loss, learning_rate

# Save model.
def saveFeedForwardNeuralNetworkToMATLABMatFile(input_size, num_hidden1_units, num_hidden2_units, num_hidden3_units, output_size):
    model_params={}
    # Hidden 1
    with tf.variable_scope('hidden1', reuse=True):
        weights = tf.get_variable('weights', [input_size, num_hidden1_units])
        biases = tf.get_variable('biases', [num_hidden1_units])
        model_params['weights_1']=weights.eval()
        model_params['biases_1']=biases.eval()
    # Hidden 2
    with tf.variable_scope('hidden2', reuse=True):
        weights = tf.get_variable('weights', [num_hidden1_units, num_hidden2_units])
        biases = tf.get_variable('biases', [num_hidden2_units])
        model_params['weights_2']=weights.eval()
        model_params['biases_2']=biases.eval()
    # Hidden 3
    with tf.variable_scope('hidden3', reuse=True):
        weights = tf.get_variable('weights', [num_hidden2_units, num_hidden3_units])
        biases = tf.get_variable('biases', [num_hidden3_units])
        model_params['weights_3']=weights.eval()
        model_params['biases_3']=biases.eval()
    # Linear (Output)
    with tf.variable_scope('linear', reuse=True):
        weights = tf.get_variable('weights', [num_hidden3_units, output_size])
        biases = tf.get_variable('biases', [output_size])
        model_params['weights_out']=weights.eval()
        model_params['biases_out']=biases.eval()
    
    return model_params

# Build the complete graph for feeding inputs, training, and saving checkpoints.
ff_nn_graph = tf.Graph()
with ff_nn_graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, D_input], name="tf_train_dataset_placeholder")
    tf_train_labels = tf.placeholder(tf.float32, shape=[batch_size, D_output], name="tf_train_labels_placeholder")
    tf_train_all_dataset = tf.constant(X_train_dataset, name="tf_train_all_dataset_constant")
    tf_valid_dataset = tf.constant(X_valid_dataset, name="tf_valid_dataset_constant")
    tf_test_dataset = tf.constant(X_test_dataset, name="tf_test_dataset_constant")
    tf_whole_dataset = tf.constant(X_init_offset_cancelled, name="tf_whole_dataset_constant")
    tf_whole_all_dataset = tf.constant(X_init_offset_cancelled_all, name="tf_whole_all_dataset_constant")
    tf_extrapolate_test_dataset = tf.constant(X_extrapt, name="tf_extrapolate_test_dataset_constant")
    
    # Currently turn off dropouts:
    tf_train_dropout_keep_prob = 0.77
    
    # Define the Neural Network model.
    defineFeedForwardNeuralNetworkModel(D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output)
    
    # Build the Prediction Graph (that computes predictions from the inference model).
    tf_outputs = performFeedForwardNeuralNetworkPrediction(tf_train_dataset, D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output, tf_train_dropout_keep_prob)
    
    # Build the Training Graph (that calculate and apply gradients).
    train_op, loss, learning_rate = performFeedForwardNeuralNetworkTraining(tf_outputs, tf_train_labels, 0.1, D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output)
#     train_op, loss, learning_rate = performFeedForwardNeuralNetworkTraining(tf_outputs, tf_train_labels, 0.00001, D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output)
    
    # Create a summary:
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("learning_rate", learning_rate)
    
    # merge all summaries into a single "operation" which we can execute in a session 
    summary_op = tf.summary.merge_all()
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf_outputs
    train_all_prediction = performFeedForwardNeuralNetworkPrediction(tf_train_all_dataset, D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output, 1.0)
    valid_prediction = performFeedForwardNeuralNetworkPrediction(tf_valid_dataset, D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output, 1.0)
    test_prediction  = performFeedForwardNeuralNetworkPrediction(tf_test_dataset, D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output, 1.0)
    whole_prediction  = performFeedForwardNeuralNetworkPrediction(tf_whole_dataset, D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output, 1.0)
    whole_all_prediction  = performFeedForwardNeuralNetworkPrediction(tf_whole_all_dataset, D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output, 1.0)
    extrapolate_test_prediction = performFeedForwardNeuralNetworkPrediction(tf_extrapolate_test_dataset, D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output, 1.0)

# Run training for num_steps and save checkpoint at the end.
with tf.Session(graph=ff_nn_graph) as session:
    # Run the Op to initialize the variables.
    tf.global_variables_initializer().run()
    print("Initialized")
    
    # create log writer object
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Start the training loop.
    for step in range(num_steps):
        # Read a batch of input dataset and labels.
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (Ct_train.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = X_train_dataset[offset:(offset + batch_size), :]
        batch_labels = Ct_train[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        # Run one step of the model.  The return values are the activations
        # from the `train_op` (which is discarded) and the `loss` Op.  To
        # inspect the values of your Ops or variables, you may include them
        # in the list passed to sess.run() and the value tensors will be
        # returned in the tuple from the call.
        _, loss_value, predictions, summary = session.run([train_op, loss, train_prediction, summary_op], feed_dict=feed_dict)
        
        # write log
        writer.add_summary(summary, step)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, loss_value))
            print("Minibatch NMSE: ", computeNMSE(predictions, batch_labels))
            print("Validation NMSE: ", computeNMSE(valid_prediction.eval(), Ct_valid))
            print("Extrapolation NMSE: ", computeNMSE(extrapolate_test_prediction.eval(), Ctt_extrapt))
        if (step % 5000 == 0):
            model_params = saveFeedForwardNeuralNetworkToMATLABMatFile(D_input, N_HIDDEN1_UNITS, N_HIDDEN2_UNITS, N_HIDDEN3_UNITS, D_output)
            print("Logging model_params.mat ...")
            sio.savemat('model_params/model_params.mat', model_params)
            
            whole_prediction_result = whole_prediction.eval()
            whole_prediction_result_dict={}
            whole_prediction_result_dict['whole_prediction_result'] = whole_prediction_result
            print("Logging Ct_fit_onset.mat ...")
            sio.savemat('scraping/Ct_fit_onset.mat', whole_prediction_result_dict)
            whole_all_prediction_result = whole_all_prediction.eval()
            whole_all_prediction_result_dict={}
            whole_all_prediction_result_dict['whole_all_prediction_result'] = whole_all_prediction_result
            print("Logging Ct_fit_all.mat ...")
            sio.savemat('scraping/Ct_fit_all.mat', whole_all_prediction_result_dict)
    print("Final Training NMSE  : ", computeNMSE(train_all_prediction.eval(), Ct_train))
    print("Final Validation NMSE: ", computeNMSE(valid_prediction.eval(), Ct_valid))
    print("Final Test NMSE      : ", computeNMSE(test_prediction.eval(), Ct_test))

Initialized
Minibatch loss at step 0: 235617.687500
Minibatch NMSE:  [ 1.1175257   1.03361785  1.00007999  1.05540442  1.00128663  1.03085387]
Validation NMSE:  [ 1.00311339  1.00260222  1.00766826  1.04254067  1.00115383  1.00440347]
Extrapolation NMSE:  [ 1.00300813  1.00572312  1.00657475  1.08814526  1.00019789  1.01724339]
Logging model_params.mat ...
Logging Ct_fit_onset.mat ...
Logging Ct_fit_all.mat ...
Minibatch loss at step 500: 288271.375000
Minibatch NMSE:  [ 0.92830443  0.92393476  1.06912315  0.98220372  0.91094601  0.97219622]
Validation NMSE:  [ 0.79589629  0.87058198  1.00832033  0.92965031  0.93063504  0.95658249]
Extrapolation NMSE:  [ 0.84826797  0.88653344  1.00056231  0.97551119  0.95926034  0.96967381]
Minibatch loss at step 1000: 379459.031250
Minibatch NMSE:  [ 0.82034498  0.84815204  1.01405978  0.95983952  0.86485511  0.86401385]
Validation NMSE:  [ 0.79715085  0.88027322  1.00164139  0.92050308  0.86801356  0.91561753]
Extrapolation NMSE:  [ 0.84787577  0.88

KeyboardInterrupt: 